# URSA Unsupervised Machine Learning for Groundwater Modeling

### Goal:  
Analyze and clean data to prepare for tensorly and kmeans clustering.  
Data has been pulled and stored in the files `./data_files/mesh.txt` and `./data_files/react.txt`

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np

# load files into DataFrames

mesh_original  = pd.read_csv("../data_files/mesh.txt", delimiter='\t', header='infer', index_col=None)
react_original = pd.read_csv("../data_files/react.txt", delimiter='\t', header='infer', index_col=None)

print(mesh_original.columns)
print(react_original.columns)

Index(['TIME                ', 'ELEM                ', 'INDEX               ',
       'P (Pa)              ', 'T (deg C)           ', 'SG                  ',
       'SS                  ', 'XNaCl               ', 'YH2OG               ',
       'XCO2aq              ', 'PCAP (Pa)           ', 'k-red               ',
       'DG (kg/m^3)         ', 'DL (kg/m^3)         ', 'PER MOD             ',
       'X2                  ', 'X3                  ', 'DX1                 ',
       'DX2                 ', 'DX3                 ', 'DX4                 ',
       'K(gas)              ', 'K(liq)              ', 'VIS(gas)            ',
       'VIS(liq)            '],
      dtype='object')
Index(['TIME                ', 'ELEM                ', 'INDEX               ',
       'Sg                  ', 'Sl                  ', 'T                   ',
       'pH                  ', 'alo2-               ', 'ca+2                ',
       'cl-                 ', 'fe+2                ', 'h+                  '

>After running the previous block it was evident that the titles had extra whitespace that needed to be removed.

In [2]:
# Whitespace Stripping Function
def df_strip(df):
  df = df.copy()
  for c in df.columns:
    df = df.rename(columns={c:c.strip()})
  return df

# before stripping
print(mesh_original.columns)
print(react_original.columns)

mesh_stripped  = df_strip(mesh_original)
react_stripped = df_strip(react_original)
print('\n---Stripping Whitespace---\n')

# after stripping
print(mesh_stripped.columns)
print(react_stripped.columns)

Index(['TIME                ', 'ELEM                ', 'INDEX               ',
       'P (Pa)              ', 'T (deg C)           ', 'SG                  ',
       'SS                  ', 'XNaCl               ', 'YH2OG               ',
       'XCO2aq              ', 'PCAP (Pa)           ', 'k-red               ',
       'DG (kg/m^3)         ', 'DL (kg/m^3)         ', 'PER MOD             ',
       'X2                  ', 'X3                  ', 'DX1                 ',
       'DX2                 ', 'DX3                 ', 'DX4                 ',
       'K(gas)              ', 'K(liq)              ', 'VIS(gas)            ',
       'VIS(liq)            '],
      dtype='object')
Index(['TIME                ', 'ELEM                ', 'INDEX               ',
       'Sg                  ', 'Sl                  ', 'T                   ',
       'pH                  ', 'alo2-               ', 'ca+2                ',
       'cl-                 ', 'fe+2                ', 'h+                  '

>Remove data that does not relate between the two files.

In [3]:
print("Unique times before fix: \nMesh: ", mesh_stripped.TIME.unique())
print("React: ", react_stripped.TIME.unique())
mesh_stripped = mesh_stripped[(mesh_stripped.TIME == 31536000) | (mesh_stripped.TIME == 102000000)]
react_stripped = react_stripped[(react_stripped.TIME == 31536000) | (react_stripped.TIME == 102000000)]
print("Mesh shape: ", mesh_stripped.shape)
print("React shape: ", react_stripped.shape)
print("\nUnique times after fix: \nMesh: ",mesh_stripped.TIME.unique())
print("React: ", react_stripped.TIME.unique())
print("Mesh shape: ", mesh_stripped.shape)
print("React shape: ", react_stripped.shape)

Unique times before fix: 
Mesh:  [3.1536e+07 1.0100e+08 1.0200e+08]
React:  [0.0000e+00 3.1536e+07 1.0200e+08]
Mesh shape:  (75642, 25)
React shape:  (75642, 36)

Unique times after fix: 
Mesh:  [3.1536e+07 1.0200e+08]
React:  [3.1536e+07 1.0200e+08]
Mesh shape:  (75642, 25)
React shape:  (75642, 36)


>Now that they are same shape, save the files.

In [4]:
# Save the fixed files
mesh_stripped.to_csv('../data_files/mesh_fixed.txt')
react_stripped.to_csv('../data_files/react_fixed.txt')

>Import the fixed files with the Index being `TIME` and `INDEX`

In [5]:
# Import as Multi-index
mesh  = pd.read_csv("../data_files/mesh_fixed.txt", delimiter=',', index_col=['TIME', 'INDEX'])
react = pd.read_csv("../data_files/react_fixed.txt", delimiter=',', index_col=['TIME', 'INDEX'])

In [9]:
print(mesh.head())
print(react.head())

                  Unnamed: 0 ELEM    P (Pa)  T (deg C)       SG  SS    XNaCl  \
TIME       INDEX                                                               
31536000.0 1               0  194  15691999  74.978610  0.00214   0  0.00382   
           2               1  195  15737867  74.980060  0.00213   0  0.00382   
           3               2  196  15782881  74.978423  0.00213   0  0.00382   
           4               3  197  15782590  74.980576  0.00213   0  0.00382   
           5               4  198  15752166  74.977743  0.00214   0  0.00382   

                    YH2OG  XCO2aq  PCAP (Pa)  ...     X2         X3  \
TIME       INDEX                              ...                     
31536000.0 1      0.00425  0.0452        0.0  ...  0.004  10.002144   
           2      0.00426  0.0452        0.0  ...  0.004  10.002134   
           3      0.00426  0.0453        0.0  ...  0.004  10.002129   
           4      0.00426  0.0453        0.0  ...  0.004  10.002131   
           5 

In [6]:
# Drop Unamed columns
mesh = mesh.drop('Unnamed: 0', 1)
react = react.drop('Unnamed: 0', 1)
# display change
print(mesh.head())
print(react.head())

                 ELEM    P (Pa)  T (deg C)       SG  SS    XNaCl    YH2OG  \
TIME       INDEX                                                            
31536000.0 1      194  15691999  74.978610  0.00214   0  0.00382  0.00425   
           2      195  15737867  74.980060  0.00213   0  0.00382  0.00426   
           3      196  15782881  74.978423  0.00213   0  0.00382  0.00426   
           4      197  15782590  74.980576  0.00213   0  0.00382  0.00426   
           5      198  15752166  74.977743  0.00214   0  0.00382  0.00426   

                  XCO2aq  PCAP (Pa)  k-red  ...     X2         X3        DX1  \
TIME       INDEX                            ...                                
31536000.0 1      0.0452        0.0      1  ...  0.004  10.002144  23.635829   
           2      0.0452        0.0      1  ...  0.004  10.002134  30.122794   
           3      0.0453        0.0      1  ...  0.004  10.002129  22.670415   
           4      0.0453        0.0      1  ...  0.004  10.0

---
### Merge the Data Frames

The Mesh and React are now ready to be merged.

In [7]:
mesh_react = pd.merge(mesh, react, how='left', on=['TIME', 'INDEX'])
print(mesh_react.head())
print(mesh_react.columns)
print(mesh_react.info())
print(mesh_react.shape)

                 ELEM_x    P (Pa)  T (deg C)       SG  SS    XNaCl    YH2OG  \
TIME       INDEX                                                              
31536000.0 1        194  15691999  74.978610  0.00214   0  0.00382  0.00425   
           2        195  15737867  74.980060  0.00213   0  0.00382  0.00426   
           3        196  15782881  74.978423  0.00213   0  0.00382  0.00426   
           4        197  15782590  74.980576  0.00213   0  0.00382  0.00426   
           5        198  15752166  74.977743  0.00214   0  0.00382  0.00426   

                  XCO2aq  PCAP (Pa)  k-red  ...        illite    albite  \
TIME       INDEX                            ...                           
31536000.0 1      0.0452        0.0      1  ... -4.440000e-07 -0.000057   
           2      0.0452        0.0      1  ... -4.440000e-07 -0.000057   
           3      0.0453        0.0      1  ... -4.440000e-07 -0.000057   
           4      0.0453        0.0      1  ... -4.420000e-07 -0.000057

---  
### Data is now merged

Save updated data to a new file called mesh_react.csv

In [20]:
#uncomment below to remove the Unnamed columns
mesh_react.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
mesh_react.to_csv('../data_files/mesh_react.csv')

---
### Clean Data

>Inspect for columns that contain all 0s or NAN

In [8]:
# loop through columns checking the sum of the column
for col in mesh_react.columns :
    if np.sum(mesh_react[col]) == 0 :
        print(col, " Sum: ", np.sum(mesh_react[col]))

SS  Sum:  0
PER MOD  Sum:  0
magnesite  Sum:  0
dolomite  Sum:  0


>The 3 columns SS, PER MOD, magnesite, and dolomite all appear to have 0 for values.  As a result, those columns will be dropped.

In [22]:
# Drop the 4 columns listed above
#mesh_react = mesh_react.drop(['SS', 'PER MOD', 'magnesite', 'dolomite'], axis=1)
#print(mesh_react.columns)

Index(['Unnamed: 0_x', 'ELEM_x', 'P (Pa)', 'T (deg C)', 'SG', 'XNaCl', 'YH2OG',
       'XCO2aq', 'PCAP (Pa)', 'k-red', 'DG (kg/m^3)', 'DL (kg/m^3)', 'X2',
       'X3', 'DX1', 'DX2', 'DX3', 'DX4', 'K(gas)', 'K(liq)', 'VIS(gas)',
       'VIS(liq)', 'Unnamed: 0_y', 'ELEM_y', 'Sg', 'Sl', 'T', 'pH', 'alo2-',
       'ca+2', 'cl-', 'fe+2', 'h+', 'h2o', 'hco3-', 'k+', 'mg+2', 'na+',
       'o2(aq)', 'sio2(aq)', 'so4-2', 'SMco2', 'Porosity', 'Permeabi.',
       'calcite', 'quartz', 'chlorite', 'illite', 'albite', 'siderite-2',
       'ankerite', 'kaolinite', 'smectite-na', 'co2(g)', 'fugcoe_co2(g)'],
      dtype='object')


> fix the columns that are considered objects

In [9]:
# Data contained spaces in values(199, 2 0, 2 1, etc.)
mesh_react.ELEM_x = mesh_react.ELEM_x.str.strip()
mesh_react.ELEM_x = mesh_react.ELEM_x.str.replace(' ', '0')
mesh_react.ELEM_x = pd.to_numeric(mesh_react.ELEM_x, errors='coerce')
print(mesh_react.ELEM_x)


mesh_react.ELEM_y = mesh_react.ELEM_y.str.strip()
mesh_react.ELEM_y = mesh_react.ELEM_y.str.replace(' ', '0')
mesh_react.ELEM_y = pd.to_numeric(mesh_react.ELEM_y, errors='coerce')
print(mesh_react.ELEM_y)

TIME         INDEX
31536000.0   1          194
             2          195
             3          196
             4          197
             5          198
             6          199
             7          200
             8          201
             9          202
             10         203
             11         204
             12         205
             13         206
             14         207
             15         208
             16         209
             17         210
             18         211
             19         286
             20         287
             21         288
             22         289
             23         290
             24         291
             25         292
             26         293
             27         294
             28         295
             29         296
             30         297
                      ...  
102000000.0  37792    61491
             37793    61492
             37794    61493
             37795    61494
 

Index(['ELEM_x', 'P (Pa)', 'T (deg C)', 'SG', 'SS', 'XNaCl', 'YH2OG', 'XCO2aq',
       'PCAP (Pa)', 'k-red', 'DG (kg/m^3)', 'DL (kg/m^3)', 'PER MOD', 'X2',
       'X3', 'DX1', 'DX2', 'DX3', 'DX4', 'K(gas)', 'K(liq)', 'VIS(gas)',
       'VIS(liq)', 'ELEM_y', 'Sg', 'Sl', 'T', 'pH', 'alo2-', 'ca+2', 'cl-',
       'fe+2', 'h+', 'h2o', 'hco3-', 'k+', 'mg+2', 'na+', 'o2(aq)', 'sio2(aq)',
       'so4-2', 'SMco2', 'Porosity', 'Permeabi.', 'calcite', 'quartz',
       'chlorite', 'illite', 'albite', 'siderite-2', 'ankerite', 'kaolinite',
       'magnesite', 'dolomite', 'smectite-na', 'co2(g)', 'fugcoe_co2(g)'],
      dtype='object')


In [12]:
print(mesh_react.head())

                  ELEM_x    P (Pa)  T (deg C)       SG  SS    XNaCl    YH2OG  \
TIME       INDEX                                                               
31536000.0 1         194  15691999  74.978610  0.00214   0  0.00382  0.00425   
           2         195  15737867  74.980060  0.00213   0  0.00382  0.00426   
           3         196  15782881  74.978423  0.00213   0  0.00382  0.00426   
           4         197  15782590  74.980576  0.00213   0  0.00382  0.00426   
           5         198  15752166  74.977743  0.00214   0  0.00382  0.00426   

                  XCO2aq  PCAP (Pa)  k-red  ...        illite    albite  \
TIME       INDEX                            ...                           
31536000.0 1      0.0452        0.0      1  ... -4.440000e-07 -0.000057   
           2      0.0452        0.0      1  ... -4.440000e-07 -0.000057   
           3      0.0453        0.0      1  ... -4.440000e-07 -0.000057   
           4      0.0453        0.0      1  ... -4.420000e-07 -0

In [13]:
print(mesh_react.columns)

Index(['ELEM_x', 'P (Pa)', 'T (deg C)', 'SG', 'SS', 'XNaCl', 'YH2OG', 'XCO2aq',
       'PCAP (Pa)', 'k-red', 'DG (kg/m^3)', 'DL (kg/m^3)', 'PER MOD', 'X2',
       'X3', 'DX1', 'DX2', 'DX3', 'DX4', 'K(gas)', 'K(liq)', 'VIS(gas)',
       'VIS(liq)', 'ELEM_y', 'Sg', 'Sl', 'T', 'pH', 'alo2-', 'ca+2', 'cl-',
       'fe+2', 'h+', 'h2o', 'hco3-', 'k+', 'mg+2', 'na+', 'o2(aq)', 'sio2(aq)',
       'so4-2', 'SMco2', 'Porosity', 'Permeabi.', 'calcite', 'quartz',
       'chlorite', 'illite', 'albite', 'siderite-2', 'ankerite', 'kaolinite',
       'magnesite', 'dolomite', 'smectite-na', 'co2(g)', 'fugcoe_co2(g)'],
      dtype='object')


In [15]:
print(mesh_react.ELEM_x, mesh_react.ELEM_y)

TIME         INDEX
31536000.0   1          194
             2          195
             3          196
             4          197
             5          198
             6          199
             7          200
             8          201
             9          202
             10         203
             11         204
             12         205
             13         206
             14         207
             15         208
             16         209
             17         210
             18         211
             19         286
             20         287
             21         288
             22         289
             23         290
             24         291
             25         292
             26         293
             27         294
             28         295
             29         296
             30         297
                      ...  
102000000.0  37792    61491
             37793    61492
             37794    61493
             37795    61494
 

In [17]:
mesh_react.to_csv('../data_files/cleaned_mesh_react.txt')